# 股票交易数据分析

In [1]:
%pwd

u'/Users/kamidox/work/stock-data'

In [3]:
import pandas as pd
import numpy as np

## 原始数据: 2000 年 - 2009 年 5 分钟历史成交数据

需要确保原始数据放在 raw 目录下，且每个年份单独一个目录。搜索 ghancn 可以免费下载 2009 年之前的数据。2009 年之后的需要购买。

In [5]:
d = pd.read_csv('raw/2008/SH600300.csv', names=['date','time','opening_price', 'ceiling_price', 'floor_price', 'closing_price', 'volume', 'amount'], header=None)

In [6]:
d.head()

,date,time,opening_price,ceiling_price,floor_price,closing_price,volume,amount
0,2008/01/02,09:35,9.89,9.90,9.76,9.77,3223.00,3174676
1,2008/01/02,09:40,9.77,9.84,9.75,9.79,2037.50,1992715
2,2008/01/02,09:45,9.78,9.82,9.78,9.79,1227.99,1203187
3,2008/01/02,09:50,9.80,9.85,9.79,9.85,1137.00,1115308
4,2008/01/02,09:55,9.84,9.95,9.84,9.94,1422.97,1405492


### 转化为日交易数据

In [7]:
g = d.groupby('date')

In [8]:
day = g.agg({'opening_price': 'first', 'ceiling_price': 'max', 'floor_price': 'min', 'closing_price': 'last', 'volume': 'sum', 'amount': 'sum'})

In [9]:
day.head()

,floor_price,opening_price,ceiling_price,volume,amount,closing_price
date,,,,,,
2008/01/02,9.75,9.89,10.31,87545.17,88759296,10.22
2008/01/03,10.22,10.31,10.88,151147.35,159856480,10.58
2008/01/04,10.28,10.58,10.76,81476.55,85745624,10.58
2008/01/07,10.45,10.48,11.09,130950.71,141612400,10.98
2008/01/08,10.30,10.95,11.00,116279.70,124193984,10.42


### 合并数据

可以参阅 stock.py 里的 `main()` 函数。把所有的数据转化为日交易数据，然后以股票代号为文件名保存在 `data` 目录下。

## 选股

什么股票是好股票？要回答这个问题，先要把最简单的问题说清楚。炒股就是低买高卖，实现获利。那么好股票的标准就是在你的习惯持股周期内，**波动最大的股票**。这很好理解吧，波动最大，我们才有可能在相对低点买入，在相对高点卖出，获利最大。

在一定的时间周期内，**衡量股票波动的指标定义为 最高价/最低价**。以我们表格中的数据，就是 ceiling_price/floor_price。这个比率最大的股票就是好股票。

关于时间周期，这个和个人的炒股习惯有关。有些人习惯做短线，可能就持股几天，或一两周。有些人习惯做长线，可能持股时长以年计算。这个完全是个人喜好问题。

有了这个思路，我们就可以玩转已经转换为日交易数据的股票，选出近期波动最大的股票。假设我们的目标是**选出一个月内波动最大的股票**。我们看一下如何用 pandas 实现这个目标。

### 过滤数据

我们先要按照考查周期来过滤数据。为了简单起见，我们假设一个月是20个交易日，且只计算交易时间，不计算停牌时间。

这里，我们直接使用青岛海尔 600690 这个股票来作为示例。我们直接读取已经合并过的数据。

In [16]:
qdhr = pd.read_csv('data/SH600300.csv')
qdhr.head()

,date,floor_price,opening_price,ceiling_price,volume,amount,closing_price
0,2000/06/30,17.33,19.99,19.99,233276,417574342,17.49
1,2000/07/03,16.45,17.46,17.46,35143,58341800,16.45
2,2000/07/04,16.46,16.48,16.99,12750,21253362,16.70
3,2000/07/05,16.45,16.70,16.71,9394,15512318,16.50
4,2000/07/06,16.00,16.40,16.50,13252,21535452,16.34


In [17]:
len(qdhr)

1997

In [32]:
import numpy as np

# 定义产生分组索引的函数
def gen_item_group_index(total, group_len):
    """ generate an item group index array 
    
    suppose total = 10, unitlen = 2, then we will return array [0 0 1 1 2 2 3 3 4 4]
    """
    
    group_count = total / group_len
    group_index = np.arange(total)
    for i in range(group_count):
        group_index[i * group_len: (i + 1) * group_len] = i
    group_index[(i + 1) * group_len : total] = i + 1
    return group_index.tolist()

gen_item_group_index(10, 3)

[0, 0, 0, 1, 1, 1, 2, 2, 2, 3]

In [50]:
group_len = 60

group_index = gen_item_group_index(len(qdhr), group_len)
# 把分组索引数据添加到股票数据里
qdhr['group_index'] = group_index
qdhr

,date,floor_price,opening_price,ceiling_price,volume,amount,closing_price,group_index
0,2000/06/30,17.33,19.99,19.99,233276.00,417574342,17.49,0
1,2000/07/03,16.45,17.46,17.46,35143.00,58341800,16.45,0
2,2000/07/04,16.46,16.48,16.99,12750.00,21253362,16.70,0
3,2000/07/05,16.45,16.70,16.71,9394.00,15512318,16.50,0
4,2000/07/06,16.00,16.40,16.50,13252.00,21535452,16.34,0
5,2000/07/07,16.09,16.36,16.50,9586.00,15617373,16.25,0
6,2000/07/10,16.22,16.27,16.41,9473.51,15438039,16.31,0
7,2000/07/11,16.28,16.33,16.58,7181.00,11761194,16.45,0
8,2000/07/12,16.31,16.38,16.97,14122.00,23457970,16.77,0
9,2000/07/13,16.62,16.78,16.98,13448.00,22619470,16.89,0


In [51]:
# 根据索引分组计算
qdhr_group = qdhr.groupby('group_index').agg({'date': 'first', 'volume': 'sum', 'floor_price': 'min', 'ceiling_price': 'max'})
qdhr_group

,date,volume,ceiling_price,floor_price
group_index,,,,
0,2000/06/30,662932.36,19.99,16.00
1,2000/09/22,294857.32,18.88,16.60
2,2000/12/22,433175.77,18.09,15.00
3,2001/04/02,919915.95,17.88,15.80
4,2001/07/03,217256.10,17.45,14.53
5,2001/09/25,214451.91,15.00,11.60
6,2001/12/25,369568.77,13.30,9.60
7,2002/04/05,253677.58,13.75,11.08
8,2002/07/08,123341.39,13.96,12.72


In [54]:
# 添加我们的波动指标 股票波动系数 = 最高价/最低价
qdhr_group['ripples_radio'] = qdhr_group.ceiling_price / qdhr_group.floor_price
qdhr_group.head()

,date,volume,ceiling_price,floor_price,ripples_radio
group_index,,,,,
0,2000/06/30,662932.36,19.99,16.00,1.249375
1,2000/09/22,294857.32,18.88,16.60,1.137349
2,2000/12/22,433175.77,18.09,15.00,1.206000
3,2001/04/02,919915.95,17.88,15.80,1.131646
4,2001/07/03,217256.10,17.45,14.53,1.200964


In [55]:
# 降序排列。我们把分组的起始日期，交易量总和都列出来，也可以观察一下交易量和股票波动比的关系
qdhr_group.sort_values('ripples_radio', ascending=False).head()

,date,volume,ceiling_price,floor_price,ripples_radio
group_index,,,,,
25,2006/11/30,12446688.62,7.97,3.72,2.142473
26,2007/03/06,14718613.80,13.26,6.66,1.990991
30,2008/03/07,3109331.03,10.74,5.60,1.917857
31,2008/06/06,2042896.94,6.84,3.79,1.804749
32,2008/09/04,5932751.04,5.32,3.12,1.705128
